# Tick-Tac-Toe
## Andrew Ribeiro 
## 11/2/2018
![Source: https://en.wikipedia.org/wiki/Futile_game](ttg.png)
In this notebook we will explore the classic game of Tick-Tac-Toe. It is a futile game, which means: if each player plays optimally, it will end in a draw. It is also a perfect information game: each opponent has complete knowledge of the state of the game. It also is a sequential game, which lends itself naturally to a tree-based representation. In addition, it is a game full of symmetry, which we will see is a key property of creating a represetation of the game where the complexity is reduced. 

## 0. Libraries

In [148]:
import numpy as np
import math
from IPython.core.display import display, HTML

## 1. Game Representation

### 1.1. Rules

Let's look at the rules that define Tick-Tac-Toe as they are customarily described: 
* A cell can either be empty, contain an X, or Y. All cells are initially empty. 
* The game board consists of 9 cells aranged in a 3x3 square. 
* A first player is selected. Players then take turns placing X's and Y's in the empty cells of the board until an end condition is met. 
* There are three end conditions: X wins, Y wins, draw. 
    * A draw is achieved when all cells are populated and neither win condition has been met. 
    * A win for either player is achieved if three of a players marks lie on a diagonal, row, or column. 

This description is sufficiently detailed to play the game, but for our purposes of modeling it, we need to translate the rules into mathematics. There are certain fixed decisions we've made in creating this game that we will consider as variables. We do this to see the game in its most general form, which will give us perspective on the particular instance of the game. 


The first of these parameters is the dimension of the board $D$. We have fixed $D = 3x3$ in the game of tick-tack-toe, but we can imagine various dimensions for the board. What if $D = 4x4$? Such a board would be consistent with the rules of the game, it would not lead to any rule becoming nonsensical. On the other hand, what if $D = 4x5$? We now have a rectangular board instead of a square. The win conditions of having peices placed along the rows or columns remain intact, but what about the diagonal win conditions? As we know, only square matricies have a diagonal. Therefore if $D$ is not a perfect square, one of our rules becomes nonsense. The diagonal rule talks about something that does not exist in rectangular squares. What about $D = 3x3x3$? Now our board is a three dimensional cube. Does this make any of our rules become non-sensical, as did our choice of a rectangular board? We could argue that a diagonal is still a valid object in the cube, but that there are just more of them in the cube. Namely we have a diagonal on every face of the cube and a diagonal through the center of the cube to each corner of the cube. What about $D = 3x3x3x3$? Our board is now a hypercube of four dimensions called a tesseract. Does a diagonal make sense in this board? This is where our intuition can break down and requires us to be more formal about how we define the win conditions of the game. 

#### The Board 
![](boardLabeling.png)
We will label the cells of the board $[1,9]$. From the rules of the game, there are three types of cell grouping that interests us: column, row, and diagonal. Let $D=mxn$ be the dimension of the board, then we can define these sets of numbers by using set builder notation: 

* Row: $\{ \{a,b,c,\ldots\} \mid (a = b + 1 = c + 2 = \ldots + \sqrt{D}-1) \wedge (a = 1+k\sqrt{D}) \wedge (0 \leq k \leq d-\sqrt{D}+1) \}$
* Column: $\{ \{f(s,1),f(s,2),f(s,3), \ldots f(s,\sqrt{D})\} \mid (1 \leq n \leq \sqrt{D}) \wedge  (1 \leq s \leq \sqrt{D}) \} \text{  Where } f(s,n) = s + \sqrt{D}(n-1)$
* Diagonal: 

#### The Cells
  
The next parameter is the domain of the cell $C$. We have fixed $C = \{∅,Y,X\}$. 

#### The Game


$\{ \{a,b,c\} \mid (a = b + 1 = c + 2) \wedge (a = 1+k\sqrt{D}) \wedge (0 \leq k \leq D-\sqrt{D}+1) \}$

In [237]:
def arithSeqs(start, step, maxVal, length):
    res = []
    for i in range(1,maxVal+1):
        if(i+step*(length-1)>maxVal):
            break;        
        res.append(list(range(i,i+step*length,step)))
    return res
    

In [280]:
for i in range(1,10):
    print( arithSeqs(1,i,16,4))
    print("")

[[1, 2, 3, 4], [2, 3, 4, 5], [3, 4, 5, 6], [4, 5, 6, 7], [5, 6, 7, 8], [6, 7, 8, 9], [7, 8, 9, 10], [8, 9, 10, 11], [9, 10, 11, 12], [10, 11, 12, 13], [11, 12, 13, 14], [12, 13, 14, 15], [13, 14, 15, 16]]

[[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 9], [4, 6, 8, 10], [5, 7, 9, 11], [6, 8, 10, 12], [7, 9, 11, 13], [8, 10, 12, 14], [9, 11, 13, 15], [10, 12, 14, 16]]

[[1, 4, 7, 10], [2, 5, 8, 11], [3, 6, 9, 12], [4, 7, 10, 13], [5, 8, 11, 14], [6, 9, 12, 15], [7, 10, 13, 16]]

[[1, 5, 9, 13], [2, 6, 10, 14], [3, 7, 11, 15], [4, 8, 12, 16]]

[[1, 6, 11, 16]]

[]

[]

[]

[]



In [238]:
arithSeqs(1,4,9,3)

[[1, 5, 9]]

In [ ]:
def columns(n):
    squareDim = math.sqrt(n)
    squareDim % 1 #1,4,7 | 2,5,8 | 3,6,9

### 1.1. Tree Representation

In [276]:
# Tick-Tac-Toe Cell 
class TTTCell:
    def __init__(self):
        self.CS = set(["∅","Y","X"])
        self.CS_e,self.CS_v = self.enumMaps(self.CS) 
        self.n = len(self.CS)
        self.state = "∅"
    
    # enumMap: ["∅","Y","X"] -> [0,1,2]
    # valueMap: [0,1,2] -> ["∅","Y","X"]
    def enumMaps(self,stateSet):
        enumMap = dict(zip(list(stateSet),range(len(stateSet))))
        valueMap = {v: k for k, v in enumMap.items()}
        return (enumMap,valueMap)
    
    def stateEnum(self,state):
        if type(state) is list:
            return list(map(lambda x: self.CS_e[x], state))
        else:
            return self.CS_e[state]
    
    def stateValue(self,enum):
        if type(enum) is list:
            return list(map(lambda x: self.CS_v[x], enum))
        else:
            return self.CS_v[enum]
    
    def sample(self,n):
        return list(map(lambda x: self.CS_v[x],list(np.random.randint(self.n, size=n))))
    
# Tick-Tac-Toe Board    
class TTTBoard:
    def isValidBoard(self, state):
        # Piece ratio constraint. 
        pr = state.count("Y") == (state.count("X") + 1) or (state.count("Y") + 1) == state.count("X") 
        
        # Win state constraint. 
        
        
        return pr
        

# Tick-Tac-Toe Game
class TTTGame: 
    def __init__(self):
        self.gameState = []
        self.CS = set(["∅","Y","X"])
        
def displayBoard( state ):
    table = "<table>{0}</table>"
    row = "<tr>{0}</tr>"
    cell = "<td>{0}</td>"
    squareDim = math.sqrt(len(state))
    
    cells = ""
    rows = "" 
    for i in range(len(state)):
        cells += cell.format(state[i])
        if(((i+1)%squareDim) == 0):
            rows += row.format(cells)
            cells = ""
    
    display(HTML(table.format(rows)))
    
def labelBoard( length,width ):
    table = "<table>{0}</table>"
    row = "<tr>{0}</tr>"
    cell = "<td>{0}</td>"
    
    cells = ""
    rows = "" 
    for i in range(1,length*width+1):
        cells += cell.format(i)
        if((i%width) == 0):
            rows += row.format(cells)
            cells = ""
    
    display(HTML(table.format(rows)))

In [277]:
c = TTTCell()
b = TTTBoard()

c9 = c.sample(9)
displayBoard(c9)
print("Is this a valid board: {0}".format(b.isValidBoard(c9)))

∅,∅,X
∅,Y,Y
Y,Y,∅


Is this a valid board: False


In [278]:
labelBoard(4,4)

1,2,3,4
5,6,7,8
9,10,11,12
13,14,15,16


In [279]:
labelBoard(3,3)

1,2,3
4,5,6
7,8,9


In [281]:
labelBoard(5,5)

1,2,3,4,5
6,7,8,9,10
11,12,13,14,15
16,17,18,19,20
21,22,23,24,25


In [139]:
list(np.random.randint(3, size=3))

[1, 1, 0]

In [78]:
CS_s = set(["∅","Y","X"])
CS_v  = np.array(list(CS))
count = np.arange(0,3)
counts = ["0","1","2"]
universe = np.zeros(9)

def cartesianProd(a,b):
    return np.transpose([np.tile(a, len(a)), np.repeat(b, len(b))])

In [51]:
np.cross(count,counts.T)

AttributeError: 'list' object has no attribute 'T'

In [66]:
cartesianProd(count,count)

array([[0, 0],
       [1, 0],
       [2, 0],
       [0, 1],
       [1, 1],
       [2, 1],
       [0, 2],
       [1, 2],
       [2, 2]])

In [82]:
def inject(universe,p,d):
    u =  np.copy(universe);
    u[p] = d
    return u

In [83]:
inject(universe,4,3)

array([0., 0., 0., 0., 3., 0., 0., 0., 0.])

In [34]:
CS = set(["∅","Y","X"])

In [42]:
dict(zip(list(CS),range(len(CS))))

{'X': 1, 'Y': 2, '∅': 0}